In [9]:
customers = [(0,1.5), (2.5,1.2), (1,1), (1,2), (1,3.4), (2,5.6), (0,1), (0,0), (4,1), (7,1), (2,3), (3,4), (4,5), (6,7), (5,9)]
facilities = [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2), (2,0), (2,1), (2,2)]
setup_cost = [4, 3, 1, 4, 7]
cost_per_mile = 1

$z_{v} = \min_{u∈U} \{c_{vu}\}, \ v∈V$

In [10]:
from itertools import product
from math import sqrt

def compute_distance(loc1, loc2):
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return sqrt(dx*dx + dy*dy)

# Compute key parameters of MIP model formulation
#num_facilities = len(facilities)
#num_customers = len(customers)
num_facilities = 5
num_customers = 6

cartesian_prod = list(product(range(num_customers), range(num_facilities)))

# Compute shipping costs
#shipping_cost = {(c,f): cost_per_mile*compute_distance(customers[c], facilities[f]) for c, f in cartesian_prod}

shipping_cost = {(0,0): 12, (0,1): 13, (0,2): 6, (0,3): 0, (0,4): 1,
(1,0): 8, (1,1): 4, (1,2): 9, (1,3): 1, (1,4): 2,
(2,0): 2, (2,1): 6, (2,2): 6, (2,3): 0, (2,4): 1,
(3,0): 3, (3,1): 5, (3,2): 2, (3,3): 10, (3,4): 8,
(4,0): 8, (4,1): 0, (4,2): 5, (4,3): 10, (4,4): 8,
(5,0): 2, (5,1): 0, (5,2): 3, (5,3): 4, (5,4): 1,}

#print('facilities :' + str(num_facilities))
#print('customers :' + str(num_customers))
#print(shipping_cost)


values = []
for i in range(0, num_customers) : values.append(0)

current_value = 1000
current_customer = 0

for key in shipping_cost:

    if key[0] != current_customer: 
        
        current_value = 100
        current_customer = key[0]

    new_value = shipping_cost[key]

    if current_value > new_value: current_value = new_value
    
    values[current_customer] = current_value

print(values)

[0, 1, 0, 2, 0, 0]


$w_{vu} = max\{0, z_v−c_{vu}\}$

In [11]:
#starting form the first customer
w = {}

def max_w(values):
    for v in range(0, num_customers):
        
        z = values[v]

        for u in range(0, num_facilities):

            c = shipping_cost.get((v, u))
            elem = max(0, z-c)
            key = (v, u)
            w[key] = elem
    return w

w = max_w(values)
print(w)

{(0, 0): 0, (0, 1): 0, (0, 2): 0, (0, 3): 0, (0, 4): 0, (1, 0): 0, (1, 1): 0, (1, 2): 0, (1, 3): 0, (1, 4): 0, (2, 0): 0, (2, 1): 0, (2, 2): 0, (2, 3): 0, (2, 4): 0, (3, 0): 0, (3, 1): 0, (3, 2): 0, (3, 3): 0, (3, 4): 0, (4, 0): 0, (4, 1): 0, (4, 2): 0, (4, 3): 0, (4, 4): 0, (5, 0): 0, (5, 1): 0, (5, 2): 0, (5, 3): 0, (5, 4): 0}


$z^{max}_s = \min_{u∈U} \{c_{su} + f_u − \sum_{v \neq s} w_{vu}\}$

In [12]:
def new_sum(s,u,w):
    
    su = 0

    for key in w:

        if key[0] == s:
            continue
        elif key[1] == u:
            su += w[key]

    return su

z_max = []
for i in range(0, num_facilities) : z_max.append(0)

for s in range(0, num_customers):

    for u in range(0, num_facilities):

        c = shipping_cost.get((s, u))
        f = setup_cost[u]
        summ = new_sum(s,u,w)
        z = c + f - summ
        z_max[u] = z
        #print(f"{c} + {f} - {summ} = {z}")
    
    values[s] = min(z_max)
    w = max_w(values)

h = len(values) # integer division in Python 3
for i in range(h):
    print(f"| {values[i]} |")
print(f"\nz = {sum(values)}")


| 4 |
| 1 |
| 0 |
| 3 |
| 3 |
| 0 |

z = 11
